In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.5
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
path = '/labs/lab07data/DO_record_per_line.json'
df = spark.read.json(path)

In [4]:
df.show()

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|
|9/humanities|15/m...|This game-based c...|  9|  en|College Foundatio...|Canvas Network|
|  14/social_sciences|What’s in your di...| 10|  en|Digital Literacies I|Canvas Network|
|  14/social_sciences|The goal of the D...| 11|  en|Digital Literacie...|Canvas Network|
|  14/social_sciences

In [11]:
df.select('lang').distinct().count()

35

In [15]:
df.groupby('lang').count().show(35)

+----+-----+
|lang|count|
+----+-----+
|  en|24553|
|  vi|    1|
|  nb|    2|
|  uz|    1|
|  ur|   11|
|  pl|    1|
|  sk|    5|
|  pt|  187|
|  sw|    1|
|  ko|    3|
|  ms|    1|
|  tr|  120|
|  de|  166|
|  es| 1374|
|  hr|    4|
|  el|    5|
|  it|   62|
|  af|    2|
|  ar|   34|
|  sv|    1|
|  nl|    6|
|  hu|    2|
|  ca|    6|
|  ru| 1231|
|  fa|    1|
|  bg|    2|
|  hi|    6|
|  et|    1|
|  zh|  169|
|  fr|  104|
|  ja|   77|
|  id|    1|
|  da|    3|
|  fi|    2|
|  he|    8|
+----+-----+



In [29]:
df = df.filter(df.lang.isin(['en', 'es', 'ru'])).cache()

In [30]:
%time
df.groupby('lang').count().show()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.68 µs
+----+-----+
|lang|count|
+----+-----+
|  en|24553|
|  es| 1374|
|  ru| 1231|
+----+-----+



### Переводим данные в pd.DatsFrames

In [33]:
import pandas as pd

In [188]:
ru_df = pd.DataFrame(df.filter(df['lang'] == 'ru').select('id', 'name', 'lang', 'desc').sort("name").take(1231), columns=['id', 'name', 'lang', 'desc'])

In [189]:
es_df = pd.DataFrame(df.filter(df['lang'] == 'es').select('id', 'name', 'lang', 'desc').sort("name").take(1374), columns=['id', 'name', 'lang', 'desc'])

In [190]:
en_df = pd.DataFrame(df.filter(df['lang'] == 'en').select('id', 'name', 'lang', 'desc').sort("name").take(24553), columns=['id', 'name', 'lang', 'desc'])

In [191]:
print (en_df.count(), es_df.count(), ru_df.count())

id      24553
name    24553
lang    24553
desc    24553
dtype: int64 id      1374
name    1374
lang    1374
desc    1374
dtype: int64 id      1231
name    1231
lang    1231
desc    1231
dtype: int64


In [227]:
ru_list_id = ru_df['id'].tolist()

In [229]:
len(ru_list_id)

1231

### Создаем списки из текстов

In [93]:
def text_list(df):
    texts = []
    for el in df.desc:
        texts.append(el)
    return texts

In [250]:
%time
ru_text = text_list(ru_df)
es_text = text_list(es_df)
en_text = text_list(en_df)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.78 µs


In [99]:
print (len(ru_text), len(es_text), len(en_text))

1231 1374 24553


### Пробуем различные лематизаторы 

In [134]:
regex = re.compile(r'[\w\d]{2,}', re.U) # пробуем токенизацию, представленную в лабе
def tokenizer_(text):
    return regex.findall(text.lower())

In [104]:
import re        # пробуем токенизацию из прошлых лекций для русского языка  (pymorphy)
GROUPING_SPACE_REGEX = re.compile(r'([^\w]|[+])', re.UNICODE)
def simple_word_tokenize(text, _split=GROUPING_SPACE_REGEX.split):
    return [t for t in _split(text.lower()) if t and not t.isspace()]

import pymorphy2
m = pymorphy2.MorphAnalyzer()

def token_r(text):
    words = simple_word_tokenize(text)
    return [m.parse(x)[0].normal_form for x in words if len(x)>=2]

In [126]:
stemmed_russian = token_r(ru_text[0])

In [127]:
stemmed_russian

['курс',
 'рассказываться',
 'продвинуть',
 'advanced',
 'алгоритм',
 'для',
 'школьник',
 'этот',
 'курс',
 'читаться',
 'на',
 'летний',
 'компьютерный',
 'школа',
 'для',
 'участник',
 'олимпиада',
 'по',
 'информатика']

### Преобразовываем тексты в вектора

### Russian

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

In [172]:
id_ru_df = ru_df[ru_df['id'].isin([1056, 1421])]
id_ru_df

In [179]:
id_ru_text = text_list(id_ru_df)
id_ru_text

In [235]:
print (len(id_ru_text), len(ru_text))

2 1231


In [258]:
ru_text982 = ru_text.copy()
ru_text982.append(id_ru_text[0])
print (len(ru_text982))

1232


In [259]:
ru_text12201 = ru_text.copy()
ru_text12201.append(id_ru_text[1])
print (len(ru_text12201))

1232


In [260]:
hcv = HashingVectorizer(tokenizer = token_r, n_features=10000)
ru_matrix982 = hcv.fit_transform(ru_text982)
print(ru_matrix982.shape)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(1232, 10000)


In [261]:
ru_matrix12201 = hcv.fit_transform(ru_text12201)
print(ru_matrix12201.shape)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(1232, 10000)


In [ ]:
sorted_d = {}

In [364]:
from collections import Counter
def matrix_to_dict(matrix, df, id_):
    cos_m = cosine_similarity(matrix[-1], matrix[:1231])
    cos_list = []
    for i in cos_m:
        for j in i:
            cos_list.append(j)
    list_id = df['id'].tolist()
    dict_ = dict(zip(list_id, cos_list))
    c = Counter(dict_)
    cnt = 0
    for i in sorted(c, key=c.get, reverse=True):
        cnt = cnt+1
        if cnt>11:
            break
        elif cnt>1:
            if id_ not in sorted_d.keys():
                sorted_d[id_] = [i]
            else:
                sorted_d[id_].append(i)
    return sorted_d

In [372]:
rr = matrix_to_dict(ru_matrix982, ru_df, '982')

In [373]:
rr = matrix_to_dict(ru_matrix12201, ru_df, '12201')

In [360]:
dd

{'982': [1260, 858, 888, 26327, 860, 13701, 22553, 778, 773, 786],
 '12201': [1236, 18331, 1001, 12725, 1273, 20288, 1252, 1332, 20351, 1365]}

### Spanish

In [338]:
id_es_df = es_df[es_df['id'].isin([16487, 23487])]
id_es_df

,id,name,lang,desc
196,16487,Aprende cómo crear gráficos en 3D con 3Ds Max ...,es,"\nEn este curso realizaremos el modelado, text..."
711,23487,"Esquemas, subtotales y análisis de datos con E...",es,Aprende a utilizar herramientas de análisis d...


In [339]:
id_es_text = text_list(id_es_df)
id_es_text

['\nEn este curso realizaremos el modelado, texturizado, iluminación y render de una grifería de estilo clásico, consiguiendo un aspecto final hiperrealista de nuestra escena con 3Ds Max y Vray.\n3dsMax es un programa de creación de gráficos y animación 3D desarrollado por Autodesk, en concreto la división Autodesk Media &amp; Entertainment (anteriormente Discreet). Creado inicialmente por el Grupo Yost para Autodesk, salió a la venta por primera vez en 1990 para DOS. 3ds Max, con su arquitectura basada en plugins, es uno de los programas de animación 3D más utilizado, especialmente para la creación de video juegos, anuncios de televisión, en arquitectura o en películas. Este curso es impartido por Daniel Bertó, dedicado a mundo del 3D desde hace más de diez años, dentro de su curriculum académico podemos destacar, un Master en Infoarquitectura, un Master en Modelado y Animación de personajes, un Master en Mudbox, así como la Carrera de Dirección de Cine Digital y efectos especiales TH

In [340]:
print (len(id_es_text), len(es_text))

2 1374


In [341]:
es_text16487 = es_text.copy()
es_text16487.append(id_es_text[0])
print (len(es_text16487))

1375


In [353]:
es_text23487 = es_text.copy()
es_text23487.append(id_es_text[1])
print (len(es_text23487))

1375


In [346]:
hcv = HashingVectorizer(tokenizer = tokenizer_, n_features=10000)
es_matrix16487 = hcv.fit_transform(es_text16487)
print(es_matrix16487.shape)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(1375, 10000)


In [354]:
es_matrix23487 = hcv.fit_transform(es_text23487)
print(es_matrix23487.shape)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(1375, 10000)


In [376]:
from collections import Counter
def matrix_to_dict(matrix, df, id_):
    cos_m = cosine_similarity(matrix[-1], matrix[:1374])
    cos_list = []
    for i in cos_m:
        for j in i:
            cos_list.append(j)
    list_id = df['id'].tolist()
    dict_ = dict(zip(list_id, cos_list))
    c = Counter(dict_)
    cnt = 0
    for i in sorted(c, key=c.get, reverse=True):
        cnt = cnt+1
        if cnt>11:
            break
        elif cnt>1:
            if id_ not in sorted_d.keys():
                sorted_d[id_] = [i]
            else:
                sorted_d[id_].append(i)
    return sorted_d

In [377]:
rr = matrix_to_dict(es_matrix16487, es_df, '16487')

In [378]:
rr = matrix_to_dict(es_matrix23487, es_df, '23487')

### English

In [381]:
id_en_df = en_df[en_df['id'].isin([21888, 26707])]
id_en_df

,id,name,lang,desc
11963,21888,Introduction to Psychology,en,"This course is an introduction to psychology, ..."
19249,26707,SQL Server - Real World Troubleshooting Skills...,en,Watch as I use one free tool to diagnose SQL ...


In [382]:
id_en_text = text_list(id_en_df)
id_en_text

['This course is an introduction to psychology, based on classic theory&nbsp; and the latest empirical research, which reviews the basic theory, methods, and research findings of scientific psychology with a strong focus on positive psychology. In this day and age, as the physical quality of life&nbsp; improves, it is particularly important to focus on psychology. In Universities in the United States, psychology is one of the most popular subjects. This course is divided into 13 lessons, an Introduction followed by 12 lessons that focus on: consciousness, sensation, perception, learning, memory, thinking, emotion, evolution, motivation, personality, social psychology, and happiness. In every lesson, the empirical studies of psychology will be given as examples, and classical theoretical perspectives dominate, all of which lead to the culminating perspective of&nbsp; positive psychology. Students will learn about the basic content and frontiers of psychology, and understand psychologica

In [383]:
print (len(id_en_text), len(en_text))

2 24553


In [384]:
en_text21888 = en_text.copy()
en_text21888.append(id_en_text[0])
print (len(en_text21888))

24554


In [385]:
en_text26707 = en_text.copy()
en_text26707.append(id_en_text[1])
print (len(en_text26707))

24554


In [393]:
from nltk.corpus import stopwords
stops = list(stopwords.words('english'))

In [394]:
hcv = HashingVectorizer(tokenizer = tokenizer_, n_features=10000, stop_words=stops)
en_matrix21888 = hcv.fit_transform(en_text21888)
print(en_matrix21888.shape)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(24554, 10000)


In [395]:
en_matrix26707 = hcv.fit_transform(en_text26707)
print(en_matrix26707.shape)

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(24554, 10000)


In [398]:
sorted_d = {'982': [1260, 858, 888, 26327, 860, 13701, 22553, 778, 773, 786], '12201': [1236, 18331, 1001, 12725, 1273, 20288, 1252, 20351, 1332, 1365], '16487': [12453, 12303, 12339, 12243, 11634, 12336, 11580, 12298, 12338, 12337], '23487': [23506, 22681, 12827, 25902, 18813, 25383, 9409, 3902, 20078, 4714]}

In [399]:
from collections import Counter
def matrix_to_dict(matrix, df, id_):
    cos_m = cosine_similarity(matrix[-1], matrix[:24553])
    cos_list = []
    for i in cos_m:
        for j in i:
            cos_list.append(j)
    list_id = df['id'].tolist()
    dict_ = dict(zip(list_id, cos_list))
    c = Counter(dict_)
    cnt = 0
    for i in sorted(c, key=c.get, reverse=True):
        cnt = cnt+1
        if cnt>11:
            break
        elif cnt>1:
            if id_ not in sorted_d.keys():
                sorted_d[id_] = [i]
            else:
                sorted_d[id_].append(i)
    return sorted_d

In [400]:
rr = matrix_to_dict(en_matrix21888, en_df, '21888')

In [401]:
rr = matrix_to_dict(en_matrix26707, en_df, '26707')

In [391]:
print (rr)

{'982': [1260, 858, 888, 26327, 860, 13701, 22553, 778, 773, 786], '12201': [1236, 18331, 1001, 12725, 1273, 20288, 1252, 20351, 1332, 1365], '16487': [12453, 12303, 12339, 12243, 11634, 12336, 11580, 12298, 12338, 12337], '23487': [23506, 22681, 12827, 25902, 18813, 25383, 9409, 3902, 20078, 4714], '21888': [343, 21721, 403, 501, 8301, 396, 17117, 21376, 5631, 19233], '26707': [26910, 26376, 7057, 13524, 13526, 13527, 13528, 19090, 19263, 8132]}


### Выгрузка

In [403]:
import json
with open('lab07s.json', 'w') as f:
    json.dump(rr, f)